Importando libs 

In [1]:
from bs4 import BeautifulSoup
import requests as r 
import pandas as pd
import calendar
import datetime
from datetime import datetime
import smtplib, ssl

Definindo URL

In [2]:
url = "https://www.metacritic.com/browse/albums/release-date/coming-soon/date"
user_agent = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

In [3]:
response = r.get(url, headers=user_agent)

In [4]:
wiki_text = response.text

In [5]:
soup = BeautifulSoup(wiki_text, 'html.parser')

Exibindo conteúdo HTML

In [6]:
print (soup)

<!DOCTYPE html>

<html lang="en">
<head>
<title>Upcoming Album Releases - Metacritic</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="View forthcoming album releases by artist on our Upcoming Music Calender at Metacritic.com" name="description"/>
<meta content="width=1024" name="viewport"/>
<meta content="Metacritic" name="application-name"/>
<meta content="#000000" name="msapplication-TileColor"/>
<meta content="/images/win8tile/76bf1426-2886-4b87-ae1c-06424b6bb8a2.png" name="msapplication-TileImage"/>
<meta content="618k3mbeki8tar7u6wvrum5lxs5cka" name="facebook-domain-verification">
<meta content="Upcoming Album Releases" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://www.metacritic.com/browse/albums/release-date/coming-soon/date" property="og:url"/>
<meta content="https://static.metacritic.com/images/icons/mc_fb_og.png" property="og:image"/>
<meta content="Metacritic" property="og:site_name"/>
<me

Abaixo, selecionaremos a tabela que queremos raspar

In [7]:
tables = soup.find_all('table')

No nosso caso, a pagina web selecionada só possui 2 tabelas. Como queremos inspecionar a primeira, setamos o argumento [0]

In [8]:
# Selecionando a tabela com os conteudos de interesse
releases = tables[0]
# Printando html da tabela desejada
print (releases.prettify())

<table class="musicTable">
 <tr class="module">
  <th class="head head_type_1" colspan="3">
   24 June 2022
  </th>
 </tr>
 <tr>
  <td class="artistName">
   <a href="/person/alexisonfire">
    Alexisonfire
   </a>
  </td>
  <td class="albumTitle">
   Otherness
  </td>
  <td class="dataComment">
  </td>
 </tr>
 <tr>
  <td class="artistName">
   CANDY
  </td>
  <td class="albumTitle">
   Heaven Is Here
  </td>
  <td class="dataComment">
  </td>
 </tr>
 <tr>
  <td class="artistName">
   <a href="/person/conan-gray">
    Conan Gray
   </a>
  </td>
  <td class="albumTitle">
   Superache
  </td>
  <td class="dataComment">
  </td>
 </tr>
 <tr>
  <td class="artistName">
   Goose
  </td>
  <td class="albumTitle">
   Dripfield
  </td>
  <td class="dataComment">
  </td>
 </tr>
 <tr>
  <td class="artistName">
   <a href="/person/jack-johnson">
    Jack Johnson
   </a>
  </td>
  <td class="albumTitle">
   Meet The Moonlight
  </td>
  <td class="dataComment">
  </td>
 </tr>
 <tr>
  <td class="artis

Função para pegar os campos de interesse e montar a dataset

In [9]:
a = []
b = []
c = []
data = pd.Series("Data")

def monta_df():
   try:
      table = releases.find_all('tr')
      
      for i in table:
         data = (i.find_all('th', {'class':'head head_type_1'}))
         # Aqui estamos pegando os dias
         for wrapper in data:
            format_data_a = wrapper.text.strip()
         a.append(format_data_a)
         # Executei esse outro for para remover um único valor duplicado de cada entrada de data
         for wrapper in data:
            format_data_a = wrapper.text.strip()
            a.remove(format_data_a)

         data2 = (i.find_all('td', {'class':'artistName'}))
         # Pegando o artista
         for wrapper in data2:
            format_data_b = wrapper.text.strip()
            b.append(format_data_b)
         
         data3 = (i.find_all('td', {'class':'albumTitle'}))
         # Pegando o album
         for wrapper in data3:
            format_data_c = wrapper.text.strip()
            c.append(format_data_c)
      
      # Concatenando as infos em um dataframe
      dict_data = {
      'Data': a,
      'Artista': b,
      'Album': c
      }
      df = pd.DataFrame(data = dict_data)
      return df
      
   except:
      print('Erro ao montar df')



Retorno do DF

In [11]:
monta_df()

,Data,Artista,Album
0,24 June 2022,Alexisonfire,Otherness
1,24 June 2022,CANDY,Heaven Is Here
2,24 June 2022,Conan Gray,Superache
3,24 June 2022,Goose,Dripfield
4,24 June 2022,Jack Johnson,Meet The Moonlight
...,...,...,...
92,7 October 2022,Bonny Light Horseman,Rolling Golden Holy
93,7 October 2022,Counterparts,A Eulogy For Those Still Here
94,7 October 2022,Lamb of God,Omens
95,21 October 2022,Dry Cleaning,Stumpwork


Função de enviar notificação

In [12]:
def enviar_email():#criei uma função para envia ro email

    port = 465  # For SSL
    smtp_server = "coloque o endereço do seu servidor smtp"
    sender_email = "coloque seu email de envio"  # Enter your address
    receiver_email = "coloque o email de quem vai receber"  # Enter receiver address
    password = "coloque sua senha"

    message = "Subject: AVISO DE LANCAMENTO NOVO AMANHA\n\n"
  
    message += "NÂO ESQUEÇA AMANHA SERÁ O LANCAMENTO DE UM NOVO ALBUM"
    
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)